## _*H2 ground state energy computation using Quantum Phase Estimation*_

This notebook demonstrates using Qiskit Chemistry to compute ground state energy of the Hydrogen (H2) molecule using QPE (Quantum Phase Estimation) algorithm. Let's look at how to carry out such computation programmatically.

This notebook has been written to use the PYSCF chemistry driver.

We first set up the H2 molecule, create the fermionic and in turn the qubit operator using PySCF.

In [1]:
from collections import OrderedDict
import time

from qiskit import BasicAer
from qiskit.transpiler import PassManager
from qiskit.aqua import AquaError
from qiskit.aqua import QuantumInstance
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.operators.op_converter import to_weighted_pauli_operator
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.aqua.algorithms import QPE
from qiskit.aqua.components.iqfts import Standard
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

distance = 0.735
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 {}'.format(distance),
                     unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
molecule = driver.run()

qubit_mapping = 'parity'
fer_op = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubit_op = Z2Symmetries.two_qubit_reduction(to_weighted_pauli_operator(fer_op.mapping(map_type=qubit_mapping, threshold=1e-10)), 2)

Couldn't find cython int routine
Couldn't find cython int routine


Using a classical exact eigenvalue solver, we can establish the reference groundtruth value of the ground state energy:

In [2]:
exact_eigensolver = ExactEigensolver(qubit_op, k=1)
result_ee = exact_eigensolver.run()
reference_energy = result_ee['energy']
print('The exact ground state energy is: {}'.format(result_ee['energy']))

The exact ground state energy is: -1.857275030202377


Next we set up the QPE algorithm instance using the HartreeFock initial state and a standard inverse quantum fourier transform, and execute:

In [3]:
num_particles = molecule.num_alpha + molecule.num_beta
two_qubit_reduction = True
num_orbitals = qubit_op.num_qubits + (2 if two_qubit_reduction else 0)

num_time_slices = 1
n_ancillae = 9

state_in = HartreeFock(qubit_op.num_qubits, num_orbitals,
                       num_particles, qubit_mapping, two_qubit_reduction)
iqft = Standard(n_ancillae)

qpe = QPE(qubit_op, state_in, iqft, num_time_slices, n_ancillae,
          expansion_mode='suzuki',
          expansion_order=2, shallow_circuit_concat=True)
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=100)
result_qpe = qpe.run(quantum_instance)
print('The ground state energy as computed by QPE is: {}'.format(result_qpe['energy']))

The ground state energy as computed by QPE is: -1.8571368753258857


As can be easily seen, the QPE computed energy is quite close to the groundtruth value we computed earlier.